In [2]:
import pandas as pd
data = pd.read_excel(r'C:\Users\cxy\OneDrive\桌面\回国\实习\8-1\17001-27000.xlsx')#,sheet_name='1001-3000'
file_path = r'C:\Users\cxy\OneDrive\桌面\回国\实习\8-1\17001-27000 医药导出.xlsx'#11:48开始
df = data.reset_index().drop(columns='index')
df['医院名称'] = df['医院名称'].astype('string')
df

,医院名称
0,潍坊口腔医院
1,寿光市口腔医院
2,寿光市皮肤病防治站
3,威海市胸科医院
4,威海市陆军医院
...,...
9995,西昌博爱医院
9996,南充博润白癜风研究院
9997,西昌建国医院
9998,西昌口腔医院


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from fuzzywuzzy import fuzz
import pandas as pd
import time

def setup_driver():
    path_to_chromedriver = r"D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"
    service = Service(executable_path=path_to_chromedriver)
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def fetch_hospital_info(query_name, driver):
    url = f'https://www.dayi.org.cn/search?keyword={query_name}'
    driver.get(url)
    time.sleep(5)

    results = driver.find_elements(By.CSS_SELECTOR, '.title-left a')
    top_results = results[:3]  # 取前五个结果
    #print(top_results)
    best_match = None
    highest_score = 0

    for result in top_results:
        hospital_name = result.text
        #print(hospital_name)
        score = fuzz.ratio(query_name, hospital_name)
        if score > highest_score:
            highest_score = score
            best_match = result
            #print(highest_score)

    if best_match and highest_score > 60:  # 可以根据需要调整阈值
        hospital_name = best_match.text
        href = best_match.get_attribute('href')
        e_id = href.split('/')[-1].split('.')[0]
        detail_url = f'https://www.dayi.org.cn/hospital/{e_id}.html'
        return query_name, hospital_name, detail_url
    else:
        return query_name, '未找到匹配的医院信息', '未找到链接'

driver = setup_driver()
#results=fetch_hospital_info("大庆油田总医院集团南区医院",driver)
results = [fetch_hospital_info(name, driver) for name in df['医院名称']]
driver.quit()  # 确保浏览器关闭
#results
# 创建结果 DataFrame 并保存
results_df = pd.DataFrame(results, columns=['查询名称', '搜索到的医院名称', 'URL'])
results_df

,查询名称,搜索到的医院名称,URL
0,潍坊口腔医院,未找到匹配的医院信息,未找到链接
1,寿光市口腔医院,未找到匹配的医院信息,未找到链接
2,寿光市皮肤病防治站,未找到匹配的医院信息,未找到链接
3,威海市胸科医院,未找到匹配的医院信息,未找到链接
4,威海市陆军医院,未找到匹配的医院信息,未找到链接
...,...,...,...
9995,西昌博爱医院,未找到匹配的医院信息,未找到链接
9996,南充博润白癜风研究院,未找到匹配的医院信息,未找到链接
9997,西昌建国医院,未找到匹配的医院信息,未找到链接
9998,西昌口腔医院,未找到匹配的医院信息,未找到链接


In [6]:
results_df1=results_df[results_df['搜索到的医院名称']!='未找到匹配的医院信息']
results_df1


,查询名称,搜索到的医院名称,URL
6,乳山市妇幼保健院,荣县妇幼保健院,https://www.dayi.org.cn/hospital/1157176.html
8,淄川区医院,淄博市淄川区医院,https://www.dayi.org.cn/hospital/1133329.html
12,宁阳县妇幼保健院,荣县妇幼保健院,https://www.dayi.org.cn/hospital/1157176.html
13,泰安市第四人民医院,会理市人民医院,https://www.dayi.org.cn/hospital/1155606.html
14,萧县中医院,平原县中医院,https://www.dayi.org.cn/hospital/1157183.html
...,...,...,...
9963,绵阳四0四医院,四川绵阳四〇四医院,https://www.dayi.org.cn/hospital/1151948.html
9966,武定县妇幼保健院,荣县妇幼保健院,https://www.dayi.org.cn/hospital/1157176.html
9977,西藏阿里地区人民医院,阿里地区人民医院,https://www.dayi.org.cn/hospital/1156428.html
9980,西藏昌都市人民医院,会理市人民医院,https://www.dayi.org.cn/hospital/1155606.html


In [7]:
results_df1=results_df[results_df['搜索到的医院名称']!='未找到匹配的医院信息']
results_df1

,查询名称,搜索到的医院名称,URL
6,乳山市妇幼保健院,荣县妇幼保健院,https://www.dayi.org.cn/hospital/1157176.html
8,淄川区医院,淄博市淄川区医院,https://www.dayi.org.cn/hospital/1133329.html
12,宁阳县妇幼保健院,荣县妇幼保健院,https://www.dayi.org.cn/hospital/1157176.html
13,泰安市第四人民医院,会理市人民医院,https://www.dayi.org.cn/hospital/1155606.html
14,萧县中医院,平原县中医院,https://www.dayi.org.cn/hospital/1157183.html
...,...,...,...
9963,绵阳四0四医院,四川绵阳四〇四医院,https://www.dayi.org.cn/hospital/1151948.html
9966,武定县妇幼保健院,荣县妇幼保健院,https://www.dayi.org.cn/hospital/1157176.html
9977,西藏阿里地区人民医院,阿里地区人民医院,https://www.dayi.org.cn/hospital/1156428.html
9980,西藏昌都市人民医院,会理市人民医院,https://www.dayi.org.cn/hospital/1155606.html


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def setup_driver():
    path_to_chromedriver = r"D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"
    service = Service(executable_path=path_to_chromedriver)
    options = webdriver.ChromeOptions()
    # 如果你想运行无头模式（不显示浏览器窗口），可以取消下一行的注释
    #ua = UserAgent()
    #user_agent = ua.random
    options.headless = True
    user_agent = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0"
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def fetch_dean_name(url):
    #driver = setup_driver()
    driver.get(url)
    time.sleep(3)
    dean_name = "院长信息未找到"  # 默认值，以防无法获取院长名称
    try:
        # 使用WebDriverWait来等待页面元素加载完成
        wait = WebDriverWait(driver, 5)  # 最多等待10秒
        # XPath to locate the dean's name following the "医院院长" label
        dean_xpath = "//p[contains(text(), '医院院长')]/following-sibling::span[@class='short-field-content']"
        dean_info = wait.until(EC.visibility_of_element_located((By.XPATH, dean_xpath)))
        dean_name = dean_info.text.strip()  # 获取院长的名字
    except Exception as e:
        print(f"Error: {e}")

    return dean_name


import pandas as pd

# 假设 `results_df` 已经存在并包含了 '查询名称', '搜索到的医院名称', 'URL' 列
# 假设 hospital_urls 是包含所有要查询的医院 URL 的列表
#hospital_urls = results_df['URL'].tolist()  # 从 results_df 获取所有的 URL
driver = setup_driver()
# 获取所有院长的名字，这一步可能需要一些时间因为要加载每一个页面
dean_names = [fetch_dean_name(url) for url in results_df1['URL']]
driver.quit()
# 将院长名字添加到现有的 DataFrame 中
results_df1['院长名称'] = dean_names  # 添加院长名称作为新的列

# 显示结果
results_df1



Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AB13EEA2+31554]
	(No symbol) [0x00007FF7AB0B7ED9]
	(No symbol) [0x00007FF7AAF7872A]
	(No symbol) [0x00007FF7AAFC8434]
	(No symbol) [0x00007FF7AAFC853C]
	(No symbol) [0x00007FF7AB00F6A7]
	(No symbol) [0x00007FF7AAFED06F]
	(No symbol) [0x00007FF7AB00C977]
	(No symbol) [0x00007FF7AAFECDD3]
	(No symbol) [0x00007FF7AAFBA33B]
	(No symbol) [0x00007FF7AAFBAED1]
	GetHandleVerifier [0x00007FF7AB448B1D+3217341]
	GetHandleVerifier [0x00007FF7AB495AE3+3532675]
	GetHandleVerifier [0x00007FF7AB48B0E0+3489152]
	GetHandleVerifier [0x00007FF7AB1EE776+750614]
	(No symbol) [0x00007FF7AB0C375F]
	(No symbol) [0x00007FF7AB0BEB14]
	(No symbol) [0x00007FF7AB0BECA2]
	(No symbol) [0x00007FF7AB0AE16F]
	BaseThreadInitThunk [0x00007FFF503D257D+29]
	RtlUserThreadStart [0x00007FFF51A0AF28+40]

Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AB13EEA2+31554]
	(No symbol) [0x00007FF7AB0B7ED9]
	(No symbol) [0x00007FF7AAF7872A]
	(No symbol) [0x00007FF7A

Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AB13EEA2+31554]
	(No symbol) [0x00007FF7AB0B7ED9]
	(No symbol) [0x00007FF7AAF7872A]
	(No symbol) [0x00007FF7AAFC8434]
	(No symbol) [0x00007FF7AAFC853C]
	(No symbol) [0x00007FF7AB00F6A7]
	(No symbol) [0x00007FF7AAFED06F]
	(No symbol) [0x00007FF7AB00C977]
	(No symbol) [0x00007FF7AAFECDD3]
	(No symbol) [0x00007FF7AAFBA33B]
	(No symbol) [0x00007FF7AAFBAED1]
	GetHandleVerifier [0x00007FF7AB448B1D+3217341]
	GetHandleVerifier [0x00007FF7AB495AE3+3532675]
	GetHandleVerifier [0x00007FF7AB48B0E0+3489152]
	GetHandleVerifier [0x00007FF7AB1EE776+750614]
	(No symbol) [0x00007FF7AB0C375F]
	(No symbol) [0x00007FF7AB0BEB14]
	(No symbol) [0x00007FF7AB0BECA2]
	(No symbol) [0x00007FF7AB0AE16F]
	BaseThreadInitThunk [0x00007FFF503D257D+29]
	RtlUserThreadStart [0x00007FFF51A0AF28+40]



C:\Users\cxy\AppData\Local\Temp\ipykernel_19816\1908674061.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df1['院长名称'] = dean_names  # 添加院长名称作为新的列


,查询名称,搜索到的医院名称,URL,院长名称
6,乳山市妇幼保健院,荣县妇幼保健院,https://www.dayi.org.cn/hospital/1157176.html,王小彬
8,淄川区医院,淄博市淄川区医院,https://www.dayi.org.cn/hospital/1133329.html,于永庆
12,宁阳县妇幼保健院,荣县妇幼保健院,https://www.dayi.org.cn/hospital/1157176.html,王小彬
13,泰安市第四人民医院,会理市人民医院,https://www.dayi.org.cn/hospital/1155606.html,梁永欣
14,萧县中医院,平原县中医院,https://www.dayi.org.cn/hospital/1157183.html,周鲁军
...,...,...,...,...
9963,绵阳四0四医院,四川绵阳四〇四医院,https://www.dayi.org.cn/hospital/1151948.html,蒋正方
9966,武定县妇幼保健院,荣县妇幼保健院,https://www.dayi.org.cn/hospital/1157176.html,王小彬
9977,西藏阿里地区人民医院,阿里地区人民医院,https://www.dayi.org.cn/hospital/1156428.html,韩军
9980,西藏昌都市人民医院,会理市人民医院,https://www.dayi.org.cn/hospital/1155606.html,梁永欣


In [9]:
results_df1.to_excel(file_path)